# Web scraping

Este notebook é uma demonstração de como extrair dados de votação de algum evento parlamentar da Câmara dos Deputados.

Você pode utulizá-lo para extrair dados das votações que seguem o mesmo padrão html da utilizada nesse exemplo.

### Documentação das bibliotecas utilizadas: 

<a href="https://docs.python-requests.org/en/latest/" title="Requests">Requests</a>

<a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/#" title="BeautifulSoup">BeautifulSoup</a>

<a href="https://pandas.pydata.org/docs/" title="Pandas">Pandas</a>

<a href="https://docs.python.org/3/library/re.html" title="Regular Expressions">Regular Expressions</a>

### Permissões:

Caso use esse algoritmo para raspagem, não utilizem urls que violem o <a href="https://www.camara.leg.br/robots.txt" title="Protocolo de Exclusão de Robôs">Protocolo de Exclusão de Robôs</a>:

- User-Agent: * 
- Disallow: /sileg/prop_lista* 
- Disallow: /internet/sileg/prop_lista* 
- Disallow: /sileg/Prop_lista* 
- Disallow: /internet/sileg/Prop_lista* 
- Disallow: /*arvore-de-apensados 
- Disallow: /proposicoesWeb/prop_arvore_tramitacoes 
- Disallow: /sileg/prop_arvore_tramitacoes 
- Disallow: /internet/deputado/Dep_Lista* 
- Disallow: /transparencia/recursos-humanos/remuneracao/*
- Disallow: /transparencia/recursos-humanos/contratos-terceirizacao* 
- Disallow: /transparencia/recursos-humanos/funcionarios* 

In [1]:
# importando as bibliotecas necessárias
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re # expressões regulares

In [2]:
def extract_html(url):
    """Extract the html code of the page.
    
    Args:
        url (str): page's url

    Returns:
        html_str (str): html extracted
    """
    
    try:
        r = requests.get(url) 
        html_str = r.text 
    except:
        raise HTTPException(404)
        
    return html_str


Abaixo temos um exemplo de um elemento da lista que contém todas as informações que precisamos por palarmentar.

    <li>
      <span class="nome">Abou Anni</span>
      <span class="nomePartido">(PSL-SP)</span>
      <span class="votou">-votou<span class="voto sim">Sim</span></span>
     </li>

Precisamos navegar até o elemento de cada deputado e extrair as informações relevantes.

In [3]:
def scrape_page(html_str):
    """Extract, assemble and returns the voting information from html in a 2D list.
    
    Args:
        html_str (str): html as a string
    
    Returns:
        vote_info (list): data scraped from html with shape (number of votants, number of features).
    
    """
    # crinado o objeto soup
    soup = BeautifulSoup(html_str) 

    # inicializa a lista
    vote_info = []
    
    # percorre o item de lista com a info:
    for elemento in soup.find_all('span', attrs={"class":"nome"}):
        lista = []
        
        lista.append(elemento.parent.contents[1].string) # nome do parlamentar
        
        party_state = re.search(r'\((.*?)-(.*?)\)',elemento.parent.contents[3].string)
        lista.append(party_state.group(1)) # partido
        lista.append(party_state.group(2)) # unidade federativa
        
        # Voto
        if not elemento.parent.select('span.votou span'): # lista vazia não tem método .string:
            lista.append('Sem voto') # ausência
        else:
            lista.append(elemento.parent.select('span.votou span')[0].string) # voto
        
        vote_info.append(lista)
        
    return vote_info
        
    

In [4]:
def save_data(vote_info, csv_name='saved'):
    """
    Convert 2D list into pandas DataFrame and store it into a .csv file.
    
    Args:
        vote_info (list): 2D list with the information gathered
        csv_name (str): name of the file to be created
    """
    df = pd.DataFrame(vote_info, columns=['nome', 'partido', 'estado', 'voto'])
    csv_name = csv_name + '.csv'
    df.to_csv(csv_name, index=False)
    return
    

In [5]:
def scraper(url, csv_name):
    """
    Recieves a url, scrape the voting information and saves it into a csv file.
    
    Args:
        url (str): page's url
        csv_name (str): name of the file to be created
    
    """
    
    html = extract_html(url)
    vote_info = scrape_page(html)
    save_data(vote_info, csv_name)
    return

In [6]:
if __name__ == '__main__':
    url = 'https://www.camara.leg.br/presenca-comissoes/votacao-portal?reuniao=62583&itemVotacao=9968&ordenacao=Nome'
    scraper(url, 'pec135')
    
    df = pd.read_csv('pec135.csv')
    
    print(df)

               nome     partido estado      voto
0    AJ Albuquerque          PP     CE  Sem voto
1    Abílio Santana          PL     BA  Sem voto
2         Abou Anni         PSL     SP       Sim
3    Acácio Favacho        PROS     AP  Sem voto
4      Adolfo Viana        PSDB     BA  Sem voto
..              ...         ...    ...       ...
504       Zé Carlos          PT     MA       Não
505         Zé Neto          PT     BA       Não
506        Zé Silva  Solidaried     MG  Sem voto
507        Zé Vitor          PL     MG       Sim
508     Zeca Dirceu          PT     PR       Não

[509 rows x 4 columns]
